In [1]:
import pandas as pd
import pycountry

In [2]:
# Reads data
df = pd.read_excel("../data/profit-shifting.xlsx")

In [3]:
# The data is long format. The avaliable indicators are below
df.indicator.unique()

array(['tax_gain_share_of_CITrev ', 'tax_loss_share_of_CITrev ',
       'profits_shifted', 'shifted_profits', 'tax_revenue_lost',
       'tax_revenue_won'], dtype=object)

In [4]:
# This are the countries that are not tax-havens. Tax-havens are under shifted_profits
df[df.indicator=='profits_shifted'].iso3.unique()

array(['AFG', 'ALB', 'ARM', 'AUS', 'AUT', 'BGD', 'BGR', 'BIH', 'BLR',
       'BOL', 'BRA', 'BWA', 'CAN', 'CHL', 'CHN', 'CMR', 'COL', 'CRI',
       'CZE', 'DEU', 'DNK', 'DOM', 'ECU', 'ESP', 'EST', 'FIN', 'FJI',
       'FRA', 'GBR', 'GEO', 'GHA', 'GRC', 'GTM', 'HND', 'HRV', 'HUN',
       'IND', 'ISL', 'ISR', 'ITA', 'JAM', 'JOR', 'JPN', 'KAZ', 'KEN',
       'KHM', 'KOR', 'KWT', 'LKA', 'LTU', 'LVA', 'MAR', 'MDA', 'MEX',
       'MKD', 'MNE', 'MOZ', 'MWI', 'NAM', 'NOR', 'NZL', 'OMN', 'PAK',
       'PER', 'PHL', 'PNG', 'POL', 'PRT', 'PRY', 'QAT', 'ROU', 'ROW',
       'RUS', 'SDN', 'SLE', 'SLV', 'SRB', 'SUR', 'SVK', 'SVN', 'SWE',
       'SYR', 'TTO', 'TUN', 'TUR', 'TZA', 'UGA', 'UKR', 'USA', 'VNM',
       'WSM', 'ZAF', 'ARG', 'EGY', 'IDN', 'MYS', 'NGA', 'THA', 'URY',
       'UZB', 'VEN', 'AZE', 'BDI', 'BEN', 'BFA', 'BRN', 'CIV', 'COD',
       'DJI', 'DMA', 'ETH', 'GMB', 'GRD', 'KGZ', 'MDG', 'MNG', 'NIC',
       'RWA', 'SEN', 'SLB', 'SWZ', 'TKM', 'MMR', 'COG'], dtype=object)

In [5]:
# Basic cleaning to remove extra space
df['indicator'] = df.indicator.str.replace("tax_loss_share_of_CITrev ","tax_loss_share_of_CITrev")

In [6]:
# Ww want to look at worldwide revenue loss
df = df[(df.counterpart=='World')]

In [7]:
# Select the countries with the biggest tax loss in 2020 among those with > 1bi lost
countries_with_over_1bi_lost = df[(df.year==2020) & (df.indicator=='tax_revenue_lost') & (df.value >= .35)].iso3.unique()
countries_with_higher_tax_loss_share = df[(df.year==2020) & 
                       (df.indicator=='tax_loss_share_of_CITrev') &
                        (df.iso3.isin(countries_with_over_1bi_lost))].sort_values(by='value', ascending=False).head(15)

countries_to_keep = countries_with_higher_tax_loss_share.iso3.unique()

In [8]:
# # Computes a percentage variation
# def percentage_variation(group):
#     '''
#     Variation over time within groups with different time ranges
#     '''
    
#     # Get the first and last observation in the group
#     first_observation = group['value'].iloc[0]
#     last_observation = group['value'].iloc[-1]
    
#     # Calculate the percentage variation
#     percentage_var = ((last_observation - first_observation) / first_observation) * 100
    
#     return percentage_var

In [9]:
### Format the table so it looks nicer
def make_dw_table_data(df, countries):
    
    # Evolution of shifted profits
    ps = df[df.iso3.isin(countries) & (df.indicator=='profits_shifted') ]
    ps = ps.pivot(index='iso3', columns='year', values='value').sort_values(by=2020, ascending=False)
    
    
    ps['iso3'] = ps.index
    ps['country'] = ps.iso3.apply(lambda x: pycountry.countries.get(alpha_3=x).name)
    ps['emoji_code'] = ps.iso3.apply(lambda x: ":" + pycountry.countries.get(alpha_3=x).alpha_2.lower() +  ":")
    ps = ps.rename(columns={ i:f"Profits booked away from country in {i}" for i in range(2015,2021)})
    
    # Makes it negative
#     for k,v in { i:f"Profits booked away from country in {i}" for i in range(2015,2021)}.items():
#         ps[v] = ps[v] * -1
        
    ps = ps.reset_index(drop=True)
        
    # Evolution of extra taxation
    trl = df[df.iso3.isin(countries) & (df.indicator=='tax_revenue_lost')]
    trl = trl.pivot(index='iso3', columns='year', values='value').sort_values(by=2020, ascending=False)
    
    trl['iso3'] = trl.index
    trl['country'] = trl.iso3.apply(lambda x: pycountry.countries.get(alpha_3=x).name)
    trl['emoji_code'] = trl.iso3.apply(lambda x: ":" + pycountry.countries.get(alpha_3=x).alpha_2.lower() +  ":")
    
    trl = trl.rename(columns={ i:f"Tax revenue lost in {i}" for i in range(2015,2021)})
    
    # Makes it negative
#     for k,v in { i:f"Tax revenue lost in {i}" for i in range(2015,2021)}.items():
#         trl[v] = trl[v] * -1
    
    trl = trl.reset_index(drop=True)
    
    
    data = ps.merge(trl)
    data['display_name'] = data['emoji_code'] + " " + data["country"]
    data['placeholder'] = "-"
    
    

    # Share of 2020 corporate tax
    citr = df[df.iso3.isin(countries) & (df.indicator=='tax_loss_share_of_CITrev') & (df.year==2020)]
    citr = citr.pivot(index='iso3', columns='year', values='value').sort_values(by=2020, ascending=False)
    citr[2020] = citr[2020] * 100
    citr = citr.rename(columns={2020: "Share of corporate tax lost in 2020"})
    citr = citr.reset_index()
        
    data = data.merge(citr)
    
    
    data = data[['display_name', 
                 'Profits booked away from country in 2015','Profits booked away from country in 2016', 
                 'Profits booked away from country in 2017', 'Profits booked away from country in 2018', 
                 'Profits booked away from country in 2019', 'Profits booked away from country in 2020',
# #                  "placeholder",
                 'Tax revenue lost in 2015','Tax revenue lost in 2016', 
                 'Tax revenue lost in 2017', 'Tax revenue lost in 2018', 
                 'Tax revenue lost in 2019',
                 'Tax revenue lost in 2020',
                 'Share of corporate tax lost in 2020']]
    
    data = data.sort_values(by='Share of corporate tax lost in 2020', ascending=False)
    
    display(data)

    
    data.to_csv("../output/datawrapper-other-countries.csv", index=False)


In [10]:
make_dw_table_data(df, countries_to_keep)

year,display_name,Profits booked away from country in 2015,Profits booked away from country in 2016,Profits booked away from country in 2017,Profits booked away from country in 2018,Profits booked away from country in 2019,Profits booked away from country in 2020,Tax revenue lost in 2015,Tax revenue lost in 2016,Tax revenue lost in 2017,Tax revenue lost in 2018,Tax revenue lost in 2019,Tax revenue lost in 2020,Share of corporate tax lost in 2020
2,:de: Germany,54.904340,64.658490,66.956890,70.946370,70.559140,55.949150,16.317570,19.216500,19.946460,21.283910,21.167740,16.784750,26.1583
1,:gb: United Kingdom,61.500900,79.621760,97.472490,102.355300,102.366300,81.170310,12.300180,15.924350,18.519770,19.447500,19.449600,15.422360,25.0603
14,:cr: Costa Rica,1.057520,0.887555,1.216996,1.432460,1.816856,1.440657,0.317256,0.266266,0.365099,0.429738,0.545057,0.432197,24.7668
11,:hu: Hungary,2.394889,3.623497,4.257021,5.341564,5.389468,4.273523,0.455029,0.688464,0.383132,0.480741,0.485052,0.384617,24.4963
13,:ng: Nigeria,NaN,3.048983,2.438911,2.992018,4.018292,3.186264,NaN,0.914695,0.731673,0.897605,1.205488,0.955879,19.6712
5,:br: Brazil,14.212060,17.105160,19.983140,18.795540,25.044110,19.858480,4.832102,5.815755,6.794266,6.390485,8.514999,6.751882,17.0762
6,:es: Spain,14.362070,14.528680,17.232930,19.650520,19.692410,15.614900,4.021380,3.632171,4.308234,4.912630,4.923103,3.903725,15.5513
8,:cl: Chile,5.181914,5.235636,5.583566,6.090758,8.511232,6.748894,1.243659,1.256553,1.423809,1.583597,2.298033,1.822201,15.2990
9,:no: Norway,5.215665,6.157640,6.004805,6.160314,7.594767,6.022193,1.408229,1.539410,1.441153,1.416872,1.670849,1.324883,15.1029
3,:fr: France,32.084130,35.599340,40.659220,39.807270,39.747920,31.517710,10.693640,11.854580,13.551720,13.136400,12.321860,8.824959,14.3509


In [11]:
### Revenue lost by emerging and developing economies

country_iso_codes = {
    'Afghanistan': 'AFG',
    'Albania': 'ALB',
    'Algeria': 'DZA',
    'Angola': 'AGO',
    'Antigua and Barbuda': 'ATG',
    'Argentina': 'ARG',
    'Armenia': 'ARM',
    'Aruba': 'ABW',
    'Azerbaijan': 'AZE',
    'The Bahamas': 'BHS',
    'Bahrain': 'BHR',
    'Bangladesh': 'BGD',
    'Barbados': 'BRB',
    'Belarus': 'BLR',
    'Belize': 'BLZ',
    'Benin': 'BEN',
    'Bhutan': 'BTN',
    'Bolivia': 'BOL',
    'Bosnia and Herzegovina': 'BIH',
    'Botswana': 'BWA',
    'Brazil': 'BRA',
    'Brunei Darussalam': 'BRN',
    'Bulgaria': 'BGR',
    'Burkina Faso': 'BFA',
    'Burundi': 'BDI',
    'Cabo Verde': 'CPV',
    'Cambodia': 'KHM',
    'Cameroon': 'CMR',
    'Central African Republic': 'CAF',
    'Chad': 'TCD',
    'Chile': 'CHL',
    'China': 'CHN',
    'Colombia': 'COL',
    'Comoros': 'COM',
    'Democratic Republic of the Congo': 'COD',
    'Republic of Congo': 'COG',
    'Costa Rica': 'CRI',
    "Côte d'Ivoire": 'CIV',
    'Djibouti': 'DJI',
    'Dominica': 'DMA',
    'Dominican Republic': 'DOM',
    'Ecuador': 'ECU',
    'Egypt': 'EGY',
    'El Salvador': 'SLV',
    'Equatorial Guinea': 'GNQ',
    'Eritrea': 'ERI',
    'Eswatini': 'SWZ',
    'Ethiopia': 'ETH',
    'Fiji': 'FJI',
    'Gabon': 'GAB',
    'The Gambia': 'GMB',
    'Georgia': 'GEO',
    'Ghana': 'GHA',
    'Grenada': 'GRD',
    'Guatemala': 'GTM',
    'Guinea': 'GIN',
    'Guinea-Bissau': 'GNB',
    'Guyana': 'GUY',
    'Haiti': 'HTI',
    'Honduras': 'HND',
    'Hungary': 'HUN',
    'India': 'IND',
    'Indonesia': 'IDN',
    'Iran': 'IRN',
    'Iraq': 'IRQ',
    'Jamaica': 'JAM',
    'Jordan': 'JOR',
    'Kazakhstan': 'KAZ',
    'Kenya': 'KEN',
    'Kiribati': 'KIR',
    'Kosovo': 'XKX',
    'Kuwait': 'KWT',
    'Kyrgyz Republic': 'KGZ',
    'Lao P.D.R.': 'LAO',
    'Lebanon': 'LBN',
    'Lesotho': 'LSO',
    'Liberia': 'LBR',
    'Libya': 'LBY',
    'Madagascar': 'MDG',
    'Malawi': 'MWI',
    'Malaysia': 'MYS',
    'Maldives': 'MDV',
    'Mali': 'MLI',
    'Marshall Islands': 'MHL',
    'Mauritania': 'MRT',
    'Mauritius': 'MUS',
    'Mexico': 'MEX',
    'Micronesia': 'FSM',
    'Moldova': 'MDA',
    'Mongolia': 'MNG',
    'Montenegro': 'MNE',
    'Morocco': 'MAR',
    'Mozambique': 'MOZ',
    'Myanmar': 'MMR',
    'Namibia': 'NAM',
    'Nauru': 'NRU',
    'Nepal': 'NPL',
    'Nicaragua': 'NIC',
    'Niger': 'NER',
    'Nigeria': 'NGA',
    'North Macedonia': 'MKD',
    'Oman': 'OMN',
    'Pakistan': 'PAK',
    'Palau': 'PLW',
    'Panama': 'PAN',
    'Papua New Guinea': 'PNG',
    'Paraguay': 'PRY',
    'Peru': 'PER',
    'Philippines': 'PHL',
    'Poland': 'POL',
    'Qatar': 'QAT',
    'Romania': 'ROU',
    'Russia': 'RUS',
    'Rwanda': 'RWA',
    'Samoa': 'WSM',
    'São Tomé and Príncipe': 'STP',
    'Saudi Arabia': 'SAU',
    'Senegal': 'SEN',
    'Serbia': 'SRB',
    'Seychelles': 'SYC',
    'Sierra Leone': 'SLE',
    'Solomon Islands': 'SLB',
    'Somalia': 'SOM',
    'South Africa': 'ZAF',
    'South Sudan': 'SSD',
    'Sri Lanka': 'LKA',
    'St. Kitts and Nevis': 'KNA',
    'St. Lucia': 'LCA',
    'St. Vincent and the Grenadines': 'VCT',
    'Sudan': 'SDN',
    'Suriname': 'SUR',
    'Syria': 'SYR',
    'Tajikistan': 'TJK',
    'Tanzania': 'TZA',
    'Thailand': 'THA',
    'Timor-Leste': 'TLS',
    'Togo': 'TGO',
    'Tonga': 'TON',
    'Trinidad and Tobago': 'TTO',
    'Tunisia': 'TUN',
    'Türkiye': 'TUR',
    'Turkmenistan': 'TKM',
    'Tuvalu': 'TUV',
    'Uganda': 'UGA',
    'Ukraine': 'UKR',
    'United Arab Emirates': 'ARE',
    'Uruguay': 'URY',
    'Uzbekistan': 'UZB',
    'Vanuatu': 'VUT',
    'Venezuela': 'VEN',
    'Vietnam': 'VNM',
    'West Bank and Gaza': 'PSE',
    'Yemen': 'YEM',
    'Zambia': 'ZMB',
    'Zimbabwe': 'ZWE'
}


In [12]:
# Profits lost by all developing countries minus ROW
df[(df.year==2020) & (df.indicator=='tax_revenue_lost') & 
   (df.iso3.isin(country_iso_codes.values())) & 
   (df.iso3!='ROW')].value.sum()

59.54941099999999

In [19]:
### Creates data for social chart.
### Remember this shows all countries that lost over 1bi, ranked by share of corporate tax
### they could be making more of
chart_df = df[df.iso3.isin(countries_to_keep) & (df.year==2019) 
  & (df.counterpart=='World') & (df.indicator=="tax_loss_share_of_CITrev")]\
    .sort_values(by='value', ascending=False)\
    .head(10)

In [21]:
chart_df['Country'] = chart_df.iso3.apply(lambda x: pycountry.countries.get(alpha_3=x).name)

In [25]:
chart_df[['Country', 'value']].to_csv("../output/datawrapper-2019-revenue-lost.csv", index=False)